Source: https://python.langchain.com/docs/tutorials/llm_chain/

In [ ]:
# Load the saved API keys

import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("You are a creative writing assistant. Help users write compelling short stories."),
    HumanMessage("Write a story about a robot learning to paint."),
]

model.invoke(messages)

## Exercise
Practice tweaking the above cells with different models, different SYSTEM messages, different Human messages

In [ ]:
messages = [
    SystemMessage("You are a coding tutor. Explain programming concepts clearly and provide simple examples."),
    HumanMessage("Explain what a for loop is and give an example in Python."),
]

model.invoke(messages)

In [ ]:
messages = [
    SystemMessage("You are a career counselor. Provide thoughtful advice about career decisions and professional development."),
    HumanMessage("I enjoy working with data and solving problems. I have a math background."),
    HumanMessage("What career paths might be a good fit for me?"),
]

model.invoke(messages)

# Streaming Message

In [ ]:
import time
for token in model.stream(messages):
    print(token.content, end="|")
    time.sleep(0.1)

## Prompt templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "You are a cooking assistant. Create a {meal_type} recipe using {ingredient} as the main ingredient"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{request}")]
)

print(prompt_template)

In [ ]:
prompt = prompt_template.invoke({"meal_type": "healthy dinner", "ingredient": "chicken", "request": "Make it easy to prepare"})

prompt

In [ ]:
prompt.to_messages()

In [ ]:
response = model.invoke(prompt)
print(response.content)

# Structured Output

https://python.langchain.com/docs/how_to/structured_output/

In [ ]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic outout structure
class JokeFormat(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(JokeFormat)

joke = structured_llm.invoke("Tell me a joke about cats")
print(joke)

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field

class BookRecommendation(BaseModel):
    """Book recommendation based on user preferences"""
    title: str = Field(description="Title of the recommended book")
    author: str = Field(description="Author of the book")
    genre: str = Field(description="Primary genre of the book")
    difficulty: str = Field(description="Reading difficulty level: beginner, intermediate, advanced")
    reasoning: str = Field(description="Why this book is recommended for the user")

structured_llm = llm.with_structured_output(BookRecommendation)

output = structured_llm.invoke("Recommend a science fiction book for someone who enjoys space exploration themes")
print(output)

In [ ]:
prompt_template = """You are a film critic assistant. Use the following movie review to answer the question. If you don't know the answer based on the review, just say that you don't know. Keep your analysis concise and insightful.
Question: {question} 
Review: {context} 
Analysis:"""

file = open(".\data\indianTales.txt")
text = file.read()

class MovieAnalysis(BaseModel):
    """A model to analyze a movie review"""
    overall_rating: str = Field(description="Overall sentiment: positive, negative, or mixed")
    key_strengths: list[str] = Field(description="Main strengths mentioned in the review")
    key_weaknesses: list[str] = Field(description="Main weaknesses mentioned in the review")
    recommendation: str = Field(description="Whether the reviewer recommends the movie")

prompt = ChatPromptTemplate([prompt_template])
msg = prompt.invoke({"question":"Analyze the overall sentiment and key points of this review.", "context":text})

structured_model = model.with_structured_output(MovieAnalysis)

output = structured_model.invoke(msg)
print(output)

In [ ]:
prompt_template = """You are a product review analyzer. Based on the given customer reviews, identify the main themes and sentiment patterns. Focus only on what customers are saying.
Customer Reviews: {reviews} 
Key Themes:"""

file = open(".\data\\animalResearch.txt")
text = file.read()

prompt = ChatPromptTemplate([prompt_template])
msg = prompt.invoke({"reviews":text})

output = llm.invoke(msg)

In [ ]:
print(output.content)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [ ]:
pages[12].page_content

# More Bigger Prompting

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

workout_template = """Create a {duration} minute workout routine for: {fitness_goal}
Include equipment needed and difficulty level.

Format your response as:

# {{Workout Name}}
--------
**Duration:** {duration} minutes  
**Goal:** {{goal}}  
**Equipment:** {{equipment list}}  
**Difficulty:** {{level}}

## Exercises:
{{exercise list with reps/sets}}
"""
question = workout_template.format(duration="30", fitness_goal="building core strength")
question

In [ ]:
response =  model.invoke(workout_template.format(duration="20", fitness_goal="improving flexibility"))

In [ ]:
from IPython.display import Markdown
Markdown(response.content)

## Example 

In [ ]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

prompt = """You are a travel planning assistant. Create a 4-day itinerary for {destination}. For each day, suggest 3 main activities or attractions in this format:

## Day 1: {{Theme}}
1. **{{Activity 1}}** - {{brief description}}
2. **{{Activity 2}}** - {{brief description}}
3. **{{Activity 3}}** - {{brief description}}
----
## Day 2: {{Theme}}
1. **{{Activity 1}}** - {{brief description}}
2. **{{Activity 2}}** - {{brief description}}
3. **{{Activity 3}}** - {{brief description}}
----
## Day 3: {{Theme}}
1. **{{Activity 1}}** - {{brief description}}
2. **{{Activity 2}}** - {{brief description}}
3. **{{Activity 3}}** - {{brief description}}
----
## Day 4: {{Theme}}
1. **{{Activity 1}}** - {{brief description}}
2. **{{Activity 2}}** - {{brief description}}
3. **{{Activity 3}}** - {{brief description}}
 
"""

response = model.invoke(prompt.format(destination="Tokyo, Japan for first-time visitors"))
from IPython.display import Markdown
Markdown(response.content)

## Example with sources

In [ ]:
prompt_template = """You are an expert technical writer. 
            
Your task is to create a short report on the topic {focus}. Divide the report into easily digestable sections. You have to use prior knowledge. While writing the report use the following instructions:
        
1. Create a report structure using markdown formatting: Create a minimum of 4 sections.
- Use ## for the section title
- Use ### for sub-section headers
        
2. Write the report following this structure:
a. Title (## header)
b. Summary (### header)
c. Sources (### header)

3. Make your title engaging based upon the focus area of the analyst: 
{focus}

4. Write a summary section:
- Set up summary with general background / context related to the focus area of the analyst
- Emphasize what is novel, interesting, or surprising about insights gathered from the interview
- Create a numbered list of source documents, as you use them
- Do not mention the names of interviewers or experts
- Aim for approximately 2000 words maximum
- Use numbered sources in your report (e.g., [1], [2]) based on information from source documents
        
5. Write a Sources section:
- Include all sources used in your report
- Provide full links to relevant websites or specific document paths
- Separate each source by a newline. Use two spaces at the end of each line to create a newline in Markdown.
- It will look like:

### Sources
- [1] Link or Document name
- [2] Link or Document name

6. Be sure to combine sources. For example this is not correct:

[3] https://ai.meta.com/blog/meta-llama-3-1/
[4] https://ai.meta.com/blog/meta-llama-3-1/

There should be no redundant sources. It should simply be:

[3] https://ai.meta.com/blog/meta-llama-3-1/
        
7. Final review:
- Ensure the report follows the required structure
- Include no preamble before the title of the report
- Check that all guidelines have been followed

"""



response = model.invoke(prompt_template.format(focus="Machine Learning in daily use operating systems"))
from IPython.display import Markdown
Markdown(response.content)

In [ ]:
prompt_template="""
Write a detailed technical report on the mathematical topic: {topic}.

Structure the report into the following sections, each clearly separated with appropriate Markdown headings (#, ##):

# Introduction

{{Give a clear, concise overview of the topic.}}

{{Explain its significance and typical applications.}}

# Mathematical Theory

{{Describe the core mathematical foundations in detail.}}

{{Include important definitions, theorems, proofs, and examples where relevant.}}

# Latest Open Problems

{{Discuss major current research challenges or unresolved questions in the field.}}

# Reference recent papers or developments (with citations).

# Future Directions

{{Suggest possible paths for future research or applications.}}

{{Discuss emerging trends and technologies that might impact this area.}}

# References

List all sources cited, preferably in a simple numbered list or BibTeX-style.

Use appropriate LaTeX-style formatting (with $$ for equations if needed) for mathematical expressions inside the Markdown.
Write in a formal and precise tone, suitable for a technical audience.
"""


response = model.invoke(prompt_template.format(topic="Machine Learning in stock predictions"))
from IPython.display import Markdown
Markdown(response.content)